### Performance Evaluation: Calculate Precision - Recall ###

In [1]:
import sys
import os
import numpy as np
import json
import pandas as pd
import logging
from pathlib import Path
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle

logger = logging.getLogger(__name__)

# PyTorch
import torch
from torchvision import ops

%load_ext autoreload
%autoreload 2
import computervision
from computervision.imageproc import is_image, ImageData, clipxywh, xyxy2xywh, xywh2xyxy, plot_boxes
from computervision.datasets import DETRdataset, get_gpu_info
from computervision.transformations import AugmentationTransform
from computervision.performance import DetectionMetrics
from computervision.inference import DETRInference

print(f'Project version: {computervision.__version__}')
print(f'Authors: {computervision.__authors__}')
print(f'Python version:  {sys.version}')

Project version: v0.0.2
Authors: The Core for Computational Biomedicine at Harvard Medical School
https://dbmi.hms.harvard.edu/about-dbmi/core-computational-biomedicine
Python version:  3.12.3 (main, Jun 18 2025, 17:59:45) [GCC 13.3.0]


In [2]:
# Check GPU availability
device, device_str = get_gpu_info()
print(f'Current device {device}')

CUDA available: True
Number of GPUs found:  1
Current device ID: 0
GPU device name:   NVIDIA GeForce RTX 3060 Laptop GPU
PyTorch version:   2.8.0a0+34c6371d24.nv25.08
CUDA version:      13.0
CUDNN version:     91200
Device for model training/inference: cuda:0
Current device cuda:0


In [3]:
# ROBOFLOW DATA
dataset = 'dataset_object_roboflow_240930'
image_dir = os.path.join(os.environ.get('DATA'), dataset)
df_file_name = 'roboflow_240930_dset.parquet'
df_file = os.path.join(image_dir, df_file_name)
df = pd.read_parquet(df_file)

file_col = 'multi_file'
pos_col = 'pos'
bbox_col = 'bbox'

# Let's get rid of the rows where we do not have positions
# And select the test data
df = df.loc[(~df[pos_col].isnull()) & (df['dset'] == 'test')].\
    reset_index(drop=True)
display(df.head(2))
df_file_list = [os.path.join(image_dir, file_name) for file_name in df[file_col].unique()]
# Check the test data
checked = [is_image(file) for file in df_file_list]
assert np.sum(checked) == len(df_file_list)
print(f'Using {len(checked)} images from the Roboflow data set')
print(f'Annotations: {df.shape[0]}')

,id,license,file_name,height,width,date_captured,file_name_hash,dset,multi_file,bbox,category,disease,pos,box_id
0,35,1,may2015-pbws-completed__7d580ea59bea307746203b...,480,640,2024-09-17T23:44:33+00:00,8f6469e5a9,test,8f6469e5a9.jpg,"[0.0, 2.0, 89.539, 213.0]",tooth 23,teeth,11,8f6469e5a9_454
1,35,1,may2015-pbws-completed__7d580ea59bea307746203b...,480,640,2024-09-17T23:44:33+00:00,8f6469e5a9,test,8f6469e5a9.jpg,"[73.0, 3.0, 195.099, 219.0]",tooth 24,teeth,12,8f6469e5a9_455


Using 48 images from the Roboflow data set
Annotations: 444


In [4]:
# Create predictions for this data set
model_name = 'rtdetr_roboflow_251005_01'
model_dir = os.path.join(os.environ.get('DATA'), 'model', model_name)
model_json_name = f'{model_name}.json'
model_json_file = os.path.join(model_dir, model_json_name)

# Load model parameters
with open(model_json_file, mode='r') as fl:
    model_parameters = json.load(fl)
display(model_parameters.keys())
model_info = model_parameters.get('model_info')

checkpoint_name = 'checkpoint-3800'
checkpoint = os.path.join(model_dir, checkpoint_name)

detr = DETRInference(device=device, checkpoint_path=checkpoint, batch_size=4)
# Convert the model outputs to the class labels (positions)
id2label = detr.model.config.id2label

# PyTorch Dataset
im_width = model_parameters.get('model_info')['im_width']
im_height = model_parameters.get('model_info')['im_height']
transforms = AugmentationTransform(im_width=im_width, im_height=im_height).\
                get_transforms(name='val')
print(*transforms, sep='\n')

dataset = DETRdataset(data=df.copy(),
                      image_processor=detr.processor,
                      image_dir=image_dir,
                      file_name_col=file_col,
                      label_id_col=None,
                      bbox_col=None,
                      transforms=transforms)

# Let's create an image id so we can match the predictions to the original inputs
file_list = dataset.file_list
file_name_list = [os.path.basename(file) for file in file_list]
file2id = dict(zip(file_name_list, range(len(file_name_list))))
df = df.assign(image_id=df[file_col].apply(lambda file: file2id.get(file)))
display(df.head(2))

# Number for each position in the test set
n_pos = df[['pos', 'image_id']].\
    groupby('pos').\
    nunique().\
    reset_index(drop=False).\
    rename(columns={'image_id': 'n_images'}).\
    sort_values(by='n_images', ascending=False).\
    reset_index(drop=True)
print()
display(n_pos.head())

dict_keys(['model_info', 'id2label', 'training_args', 'processor_params', 'bbox_format'])

NoOp(p=1.0)
AutoContrast(p=1.0, cutoff=0.0, ignore=None, method='cdf')
CLAHE(p=1.0, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


,id,license,file_name,height,width,date_captured,file_name_hash,dset,multi_file,bbox,category,disease,pos,box_id,image_id
0,35,1,may2015-pbws-completed__7d580ea59bea307746203b...,480,640,2024-09-17T23:44:33+00:00,8f6469e5a9,test,8f6469e5a9.jpg,"[0.0, 2.0, 89.539, 213.0]",tooth 23,teeth,11,8f6469e5a9_454,0
1,35,1,may2015-pbws-completed__7d580ea59bea307746203b...,480,640,2024-09-17T23:44:33+00:00,8f6469e5a9,test,8f6469e5a9.jpg,"[73.0, 3.0, 195.099, 219.0]",tooth 24,teeth,12,8f6469e5a9_455,0


,pos,n_images
0,3,28
1,4,28
2,31,28
3,2,27
4,30,27


In [5]:
# Run the predictions
threshold = 0.05
pred_df = detr.predict_on_dataset(dataset=dataset, threshold=threshold)

# We need the position labels
pred_df[pos_col] = pred_df['category_id'].apply(lambda cl: id2label.get(cl))
pred_df = pred_df.astype({pos_col: int})
display(pred_df.head(2))

Predicting batch 10 of 12.


,image_id,image_width,image_height,batch,category_id,bbox,score,area,pos
0,0,640,480,0,18,"[283, 184, 356, 175]",0.983528,62300,19
1,0,640,480,0,19,"[143, 185, 292, 174]",0.976288,50808,20


### Classify the outputs for all test images ###

In [64]:
# IoU threshold
iou_threshold = 0.5

# LOOP OVER ALL TEST IMAGES
missed_df_list = []
cl_df_list = []

for idx, image_id in enumerate(image_id_list):
    
    df_id = df.loc[(df['image_id'] == image_id)]
    file = os.path.join(image_dir, df_id[file_col].tolist()[0])
    im = ImageData().load_image(file)
    im_width, im_height = im.shape[1], im.shape[0]
    
    # True bboxes
    true_labels = df_id[pos_col].tolist()
    true_bboxes = df_id[bbox_col].tolist()
    true_bboxes = [clipxywh(list(box), xlim=(0, im_width), ylim=(0, im_height), decimals=0) for box in true_bboxes]
    
    # Pred bboxes
    pred_df_id = pred_df.loc[(pred_df['image_id'] == image_id)]
    pred_labels = pred_df_id[pos_col].tolist()
    pred_bboxes = pred_df_id['bbox'].tolist()
    pred_bboxes = [clipxywh(list(box), xlim=(0, im_width), ylim=(0, im_height), decimals=0) for box in pred_bboxes]
    
    # Classification
    missed_idx, cl_df_idx = DetectionMetrics(im_width=im_width, im_height=im_height).\
        classify_predictions(true_labels=true_labels,
                             true_bboxes=true_bboxes,
                             pred_labels=pred_labels,
                             pred_bboxes=pred_bboxes,
                             iou_threshold=iou_threshold)
    cl_df_idx = cl_df_idx.assign(score=pred_df_id['score'].tolist(),
                                 image_id=image_id)
    missed_idx = pd.DataFrame({pos_col:missed_idx})
    missed_idx = missed_idx.assign(image_id=image_id)
    
    cl_df_list.append(cl_df_idx)
    missed_df_list.append(missed_idx)

cl_df = pd.concat(cl_df_list, ignore_index=True)
missed_df = pd.concat(missed_df_list, ignore_index=True)

[11, 13]
